# Clean Core Images

This notebook demonstrates how to clean and process core images using GeoSuite's imaging module.

## Overview

This notebook will show you how to:

1. Load core images
2. Apply image cleaning techniques
3. Enhance image quality
4. Prepare images for further analysis

In [ ]:
# Import GeoSuite modules
import numpy as np
import matplotlib.pyplot as plt

try:
    from geosuite.imaging import core_processing
    IMAGING_AVAILABLE = True
    print("GeoSuite imaging module imported successfully!")
except ImportError:
    IMAGING_AVAILABLE = False
    print("Note: Imaging module requires scikit-image. Install with: pip install scikit-image")
    print("Creating synthetic image for demonstration...")
    
    # Create synthetic core image for demonstration
    synthetic_image = np.random.randint(0, 255, (500, 300, 3), dtype=np.uint8)
    print(f"Created synthetic image: {synthetic_image.shape}")

In [ ]:
# Example: Load and display core image
# In practice: image = plt.imread('path/to/core/image.jpg')

if IMAGING_AVAILABLE:
    print("Use core_processing functions to clean and process core images")
    print("Functions available:")
    print("  - clean_core_image(): Remove artifacts and enhance quality")
    print("  - enhance_contrast(): Improve image contrast")
else:
    # Display synthetic image
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.imshow(synthetic_image)
    ax.set_title('Synthetic Core Image (Demo)')
    ax.axis('off')
    plt.tight_layout()
    plt.show()
    print("\nNote: Replace with actual core image processing code when imaging module is available")

## Summary

This notebook demonstrated:

-  Loading core images
-  Using GeoSuite imaging functions (when available)

### Next Steps

- Install scikit-image for full imaging capabilities: `pip install geosuite[imaging]`
- Load your own core images
- Apply cleaning and enhancement algorithms
- Process images for automated analysis